In [1]:
from floweaver import *
import pandas as pd
import numpy as np

In [2]:
flows = pd.read_excel('../data/glass_paper.xlsx', sheet_name='Flows', usecols = ['source', 'target', 'type', 'value'])
dataset = Dataset(flows)
flows

,source,target,type,value
0,RM,FGM,Soda Ash,18.776050
1,RM,FGM,Silica,53.219756
2,RM,FGM,Limestone,11.788444
3,NaN,NaN,NaN,NaN
4,RM,CGM,Soda Ash,13.587764
5,RM,CGM,Silica,38.513822
6,RM,CGM,Limestone,8.531006
7,CGM,CGF,Liquid glass,87.611504
8,CGM,loss1,Emission,9.379862
9,FGM,FGF,Liquid glass,83.320973


In [3]:
nodes = {
    # Raw materials
    "RME": ProcessGroup(["RM"], Partition.Simple("type", ['Silica', 'Limestone', 'Soda Ash',]), 
                        title = 'Raw material extraction'),
     
    # Glass melting
    "FGM": ProcessGroup(["FGM"], title = 'Flat glass melting'),
    "CGM": ProcessGroup(["CGM"], title = 'Container glass melting'),
    "emissions": ProcessGroup(["loss1", "loss2"], title = 'Process emissions'),
    
    # Glass forming
    "FGF": ProcessGroup(["FGF"], title = 'Flat glass forming'),
    "CGF": ProcessGroup(["CGF"], title = 'Container glass forming'),
   
    # Glass use
    "FGU": ProcessGroup(["FGU"], Partition.Simple("type", 
                                                  ['Buildings', 'Automotive', 'Other FG',]), 
                        title = 'Use of flat glass'),
    
    "CGU": ProcessGroup(["CGU"], Partition.Simple("type", 
                                                  ['Beverages', 'Food', 'Other CG',]), 
                        title = 'Use of containers'),
    
    # End of life
    "EoL": ProcessGroup(["EoL"], Partition.Simple("type", 
                                                  ['Buildings', 'Automotive', 'Other FG',] + 
                                                  ['Beverages', 'Food', 'Other CG',]), 
                                                  title = 'Disposal'),
        
    'scrap1': Waypoint(title='', direction='L'),
    'scrap2': Waypoint(title='', direction='L'),
    'scrap3': Waypoint(title='', direction='L'),

}

bundles = [
    Bundle("RME", "FGM"),
    Bundle("RME", "CGM"),
    
    Bundle("FGM", "FGF"),
    Bundle("CGM", "CGF"),
        
    Bundle("FGM", "emissions"),
    Bundle("CGM", "emissions"),
    
    Bundle("CGF", "CGU"),
    Bundle("CGF", "CGM", waypoints = ['scrap2']),

    Bundle("FGF", "FGU"),
    Bundle("FGF", "FGM", waypoints = ['scrap1']),
    
    Bundle("CGU", "CGM", waypoints = ['scrap3']),
    
    Bundle("FGU", "EoL"),
    Bundle("CGU", "EoL"),
    
]

ordering = [
    [[],            ["RME"],                                      ],
    [[],            ["FGM", 'scrap1', "CGM", 'scrap2', 'scrap3'], ],
    [['emissions'], ["FGF", "CGF"],                               ],
    [[],            ['FGU', "CGU"],                               ],
    [[],            ['EoL', ],                                    ],
]

flow_partition = Partition.Simple("type",
        np.unique(flows['type'].to_list())
    )

sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition = flow_partition)
weave(sdd, dataset, palette='Paired_12').to_widget(width=1150, height = 600).auto_save_svg('../outputs/global_glass_sankey.svg')

SankeyWidget(groups=[{'id': 'RME', 'type': 'process', 'title': 'Raw material extraction', 'nodes': ['RME^Silic…